# Churn Prediction (Fuga de Clientes)

Este notebook aborda la predicción de fuga de clientes en banca. Puedes usar datos generados sintéticamente o cargar un archivo CSV externo.

## 1. Importar librerías

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## 2. Cargar datos
### A) Generar datos sintéticos

In [ ]:
np.random.seed(42)
n = 1000
data = pd.DataFrame({
    'antiguedad': np.random.randint(1, 10, n),
    'productos': np.random.randint(1, 5, n),
    'transacciones_mensuales': np.random.poisson(20, n),
    'satisfaccion': np.random.uniform(1, 5, n),
    'segmento': np.random.choice(['Retail','Premium','Pyme'], n),
})
# Generar variable objetivo (churn)
data['churn'] = (0.3*(data['antiguedad']<3) + 0.3*(data['satisfaccion']<2.5) + np.random.rand(n)) > 0.7
data['churn'] = data['churn'].astype(int)
data.head()

### B) Cargar datos desde CSV
> Si tienes tu propio archivo, descomenta la línea de abajo y reemplaza la ruta.

In [ ]:
# data = pd.read_csv('clientes_churn.csv')
# data.head()

## 3. Análisis exploratorio y visualizaciones

In [ ]:
print(data['churn'].value_counts())
data.describe()
data.groupby('segmento')['churn'].mean().plot(kind='bar')
plt.title('Tasa de churn por segmento')
plt.ylabel('Churn Rate')
plt.show()

## 4. Preprocesamiento

In [ ]:
X = data.drop('churn', axis=1)
y = data['churn']
X['segmento'] = LabelEncoder().fit_transform(X['segmento'])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

## 5. Modelos
### A) Regresión Logística

In [ ]:
lr = LogisticRegression(max_iter=200)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print(classification_report(y_test, y_pred_lr))

### B) XGBoost

In [ ]:
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

## 6. Evaluación y visualización

In [ ]:
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plt.title('Confusión LR')
plt.imshow(confusion_matrix(y_test, y_pred_lr), cmap='Blues')
plt.subplot(1,2,2)
plt.title('Confusión XGBoost')
plt.imshow(confusion_matrix(y_test, y_pred_xgb), cmap='Greens')
plt.show()
fpr_lr, tpr_lr, _ = roc_curve(y_test, lr.predict_proba(X_test)[:,1])
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, xgb_model.predict_proba(X_test)[:,1])
plt.plot(fpr_lr, tpr_lr, label='LogisticReg')
plt.plot(fpr_xgb, tpr_xgb, label='XGBoost')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.legend()
plt.title('Curvas ROC')
plt.show()

## 7. Importancia de variables
### XGBoost

In [ ]:
xgb.plot_importance(xgb_model)
plt.show()

## 8. Comentarios finales

- Puedes modificar los parámetros de los modelos para tuning.
- Prueba con tu propio CSV o ajusta las variables sintéticas.
- Agrega nuevas visualizaciones para enriquecer el análisis.